In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar
%matplotlib inline
import matplotlib.pyplot as plt
from rtree import index

#import models
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
#from xgboost import XGBRegressor
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import StrMethodFormatter

from sklearn import model_selection
from sklearn.metrics import explained_variance_score
# from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from tabulate import _table_formats, tabulate
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import pygad
import time
import datetime

from sklearn import metrics

In [6]:
datas = './MyData.xlsx'

In [7]:
df=pd.read_excel(datas)

In [8]:
df

,type,cell,aperture,aperture2,ExpansionRatio,M-N,Μ-N_lff,Μ-N_kFF,ionicRad,MetalNum,...,size - kinetic diameter (Å),diffusivity,solubility,permeability,Unnamed: 39,Unnamed: 40,Unnamed: 41,aperture2.1,Dresults,number of modifications
0,CdIF-1,1.8077,3.92,3.93000,1.002551,2.18,2.257,45354.60,92,48,...,2.60,6.000000e-08,NaN,NaN,NaN,NaN,NaN,888.0,951.0,72.0
1,CdIF-1,1.8077,3.92,3.99000,1.017857,2.18,2.257,45354.60,92,48,...,2.89,5.000000e-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CdIF-1,1.8077,3.92,4.02000,1.025510,2.18,2.257,45354.60,92,48,...,3.46,1.000000e-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CdIF-1,1.8077,3.92,3.97000,1.012755,2.18,2.257,45354.60,92,48,...,3.30,4.000000e-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CdIF-1,1.8077,3.92,4.04150,1.030995,2.18,2.257,45354.60,92,48,...,3.64,5.000000e-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1136,dIm_Mg,NaN,2.95,NaN,0.000000,NaN,2.139,38994.88,71,12,...,4.42,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1137,dIm_Mg,NaN,2.95,3.12791,1.060308,NaN,2.139,38994.88,71,12,...,3.69,4.910000e-12,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1138,dIm_Mg,NaN,2.95,3.17050,1.074746,NaN,2.139,38994.88,71,12,...,4.10,1.190000e-16,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1139,dIm_Mg,NaN,2.95,NaN,0.000000,NaN,2.139,38994.88,71,12,...,5.10,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.drop(df[df['type'] == 'dFm_Be'].index, inplace = True)

In [12]:
df.drop(df[df['type'] == 'Cd-I-ZIF-7-8'].index, inplace = True)

In [13]:
df

,type,cell,aperture,aperture2,ExpansionRatio,M-N,Μ-N_lff,Μ-N_kFF,ionicRad,MetalNum,...,size - kinetic diameter (Å),diffusivity,solubility,permeability,Unnamed: 39,Unnamed: 40,Unnamed: 41,aperture2.1,Dresults,number of modifications
0,CdIF-1,1.8077,3.92,3.93000,1.002551,2.18,2.257,45354.60,92,48,...,2.60,6.000000e-08,NaN,NaN,NaN,NaN,NaN,888.0,951.0,72.0
1,CdIF-1,1.8077,3.92,3.99000,1.017857,2.18,2.257,45354.60,92,48,...,2.89,5.000000e-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CdIF-1,1.8077,3.92,4.02000,1.025510,2.18,2.257,45354.60,92,48,...,3.46,1.000000e-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CdIF-1,1.8077,3.92,3.97000,1.012755,2.18,2.257,45354.60,92,48,...,3.30,4.000000e-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CdIF-1,1.8077,3.92,4.04150,1.030995,2.18,2.257,45354.60,92,48,...,3.64,5.000000e-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1136,dIm_Mg,NaN,2.95,NaN,0.000000,NaN,2.139,38994.88,71,12,...,4.42,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1137,dIm_Mg,NaN,2.95,3.12791,1.060308,NaN,2.139,38994.88,71,12,...,3.69,4.910000e-12,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1138,dIm_Mg,NaN,2.95,3.17050,1.074746,NaN,2.139,38994.88,71,12,...,4.10,1.190000e-16,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1139,dIm_Mg,NaN,2.95,NaN,0.000000,NaN,2.139,38994.88,71,12,...,5.10,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# create excel writer object
writer = pd.ExcelWriter('./TrainData.xlsx')
# write dataframe to excel
df.to_excel(writer)
# save the excel
writer.save()
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


C:\Users\pkrok\AppData\Local\Temp\ipykernel_26584\2135609772.py:6: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()
